# Phase 2 - Week 3 - Day 3 PM - ETL

## A. Setup Database in MongoDB

In [ ]:
# Install package pymongo

# !pip install -q pymongo

import pymongo
pymongo.__version__

'4.12.0'

In [ ]:
# Get current IP Adress

!curl ipecho.net/plain

36.70.15.52


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    11  100    11    0     0     36      0 --:--:-- --:--:-- --:--:--    36


In [ ]:
# Define URI

URI = 'YOUR-URI'

In [ ]:
# Test connection

client = pymongo.MongoClient(URI, server_api=pymongo.server_api.ServerApi(version="1", strict=True, deprecation_errors=True))

try:
  client.admin.command("ping")
  print("Connected successfully")

except Exception as e:
  raise Exception("The following error occurred: ", e)

Connected successfully


In [ ]:
# Get all databases

client.list_database_names()

['my_store', 'admin', 'local']

In [ ]:
# Create a new database

db_socmed = client['social_media']

In [ ]:
# Create a new collection

coll_raw_data = db_socmed['raw_data']

## B. Insert into MongoDB

In [ ]:
# Load dataset

import json
from pprint import pprint

with open('raw_data.json', 'r') as f:
    all_data = json.load(f)

pprint(all_data)

[{'conn_info': {'browser': 'Mozilla',
                'platform': 'iPhone',
                'status_code': 200,
                'token': '229a6669'},
  'destination_id': '0',
  'message_text': 'BIG SALE ALERT! Diskon hingga 50% untuk semua produk kami, '
                  'hanya sampai hari Minggu ini! Buruan, sebelum kehabisan! '
                  '#Promo #DiskonGede #Sale',
  'origin_id': 'admin_1',
  'post_id': 'ER3020',
  'post_time': '16/02/2025, 06:28:12',
  'post_type': 'post'},
 {'conn_info': {'browser': 'Safari',
                'platform': 'Macintosh',
                'status_code': 200,
                'token': '03f2395f'},
  'destination_id': '0',
  'message_text': 'BIG SALE ALERT! Diskon hingga 50% untuk semua produk kami, '
                  'hanya sampai hari Minggu ini! Buruan, sebelum kehabisan! '
                  '#Promo #DiskonGede #Sale',
  'origin_id': 'admin_3',
  'post_id': 'HT1988',
  'post_time': '16/02/2025, 07:34:18',
  'post_type': 'post'},
 {'conn_info': {

In [ ]:
# Insert raw data into a MongoDB Collection

coll_raw_data.insert_many(all_data)

InsertManyResult([ObjectId('67fbdc09b33ebaf1a928555b'), ObjectId('67fbdc09b33ebaf1a928555c'), ObjectId('67fbdc09b33ebaf1a928555d'), ObjectId('67fbdc09b33ebaf1a928555e'), ObjectId('67fbdc09b33ebaf1a928555f'), ObjectId('67fbdc09b33ebaf1a9285560'), ObjectId('67fbdc09b33ebaf1a9285561'), ObjectId('67fbdc09b33ebaf1a9285562'), ObjectId('67fbdc09b33ebaf1a9285563'), ObjectId('67fbdc09b33ebaf1a9285564'), ObjectId('67fbdc09b33ebaf1a9285565'), ObjectId('67fbdc09b33ebaf1a9285566'), ObjectId('67fbdc09b33ebaf1a9285567'), ObjectId('67fbdc09b33ebaf1a9285568'), ObjectId('67fbdc09b33ebaf1a9285569'), ObjectId('67fbdc09b33ebaf1a928556a'), ObjectId('67fbdc09b33ebaf1a928556b'), ObjectId('67fbdc09b33ebaf1a928556c'), ObjectId('67fbdc09b33ebaf1a928556d'), ObjectId('67fbdc09b33ebaf1a928556e')], acknowledged=True)

## C. Create Database and Table

### C.1 - Setup

First, you need to create a database named `social_media`. You can do this using the SQL syntax below, or manually by right-clicking on your server and selecting `Create` and then click `Database`.

```sql
CREATE DATABASE social_media;
```

In [ ]:
# Import library

import pandas as pd
import psycopg2

In [ ]:
# Create connection to PostgreSQL
conn = psycopg2.connect(
    dbname="social_media",
    user="",
    password="",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

### C.2 - Table `interaction`

Create a table named `interaction` that shows statistics on the number of interactions each user has with an admin. Sort the results from highest to lowest based on the number of interactions. Example table format:

| origin_id | destination_id | total_interaction |
| --- | --- | --- |
| user_1 | admin_3 | 3 |
| user_1 | admin_2 | 3 |
| user_2 | admin_3 | 1 |

Based on the available data, every post that involves interaction between a user and an admin has a `post_type = 'mention'`. On the other hand, posts with a `post_type = 'post'` are made by admins without any interaction, as indicated by `destination_id = 0`.

In [ ]:
# Step 1 : Extract (Find multiple documents from MongoDB collection)

results = coll_raw_data.find(
    {'post_type': 'mention'},
    {'_id': 0, 'origin_id': 1, 'destination_id': 1})

results = list(results)
results

[{'origin_id': 'user_6', 'destination_id': 'admin_1'},
 {'origin_id': 'user_7', 'destination_id': 'admin_1'},
 {'origin_id': 'user_1', 'destination_id': 'admin_2'},
 {'origin_id': 'user_5', 'destination_id': 'admin_1'},
 {'origin_id': 'user_3', 'destination_id': 'admin_1'},
 {'origin_id': 'user_7', 'destination_id': 'admin_1'},
 {'origin_id': 'user_5', 'destination_id': 'admin_3'},
 {'origin_id': 'user_8', 'destination_id': 'admin_1'},
 {'origin_id': 'user_1', 'destination_id': 'admin_3'}]

In [ ]:
# Step 2 : Transform (Count the number of interactions based on `origin_id`` and `destination_id`)

df = pd.DataFrame(results)
interaction_df = (
    df.groupby(['origin_id', 'destination_id'])
      .size()
      .reset_index(name='total_interaction')
      .sort_values(by='total_interaction', ascending=False)
      .reset_index(drop=True)
)
interaction_df

,origin_id,destination_id,total_interaction
0,user_7,admin_1,2
1,user_1,admin_2,1
2,user_1,admin_3,1
3,user_3,admin_1,1
4,user_5,admin_1,1
5,user_5,admin_3,1
6,user_6,admin_1,1
7,user_8,admin_1,1


In [ ]:
# Step 3 : Load (Create table `interaction` and insert the data)

## Create table `interaction`
cur.execute("""
    CREATE TABLE IF NOT EXISTS interaction (
        origin_id VARCHAR(10),
        destination_id VARCHAR(10),
        total_interaction INTEGER
    );
""")

## Insert the data
for index, row in interaction_df.iterrows():
    print('Insert data - row : ', index)
    cur.execute(
        """
        INSERT INTO interaction (origin_id, destination_id, total_interaction)
        VALUES (%s, %s, %s);
        """,
        (row['origin_id'], row['destination_id'], int(row['total_interaction'])))

conn.commit()

Insert data - row :  0
Insert data - row :  1
Insert data - row :  2
Insert data - row :  3
Insert data - row :  4
Insert data - row :  5
Insert data - row :  6
Insert data - row :  7


### C.3 - Table `admin_first_message`

Create a table named `admin_first_message` that contains the first post made by each admin. Sort the results by admin name (`origin_id`). Example table format:

| post_id | post_time | origin_id | message_text |
| --- | --- | --- | --- |
| XX1113 | '16/02/2025, 10:19:44' | admin_1 | Text |
| XX1112 | '16/02/2025, 08:44:21' | admin_2 | Text |
| XX1111 | '16/02/2025, 09:12:00' | admin_3 | Text |

In [ ]:
# Step 1 : Extract (Find all posts from all admins)

results = coll_raw_data.find(
    {'post_type': 'post'},
    {'_id': 0, 'post_id':1, 'post_time':1, 'origin_id': 1, 'message_text': 1})

results = list(results)
results

[{'post_id': 'ER3020',
  'post_time': '16/02/2025, 06:28:12',
  'origin_id': 'admin_1',
  'message_text': 'BIG SALE ALERT! Diskon hingga 50% untuk semua produk kami, hanya sampai hari Minggu ini! Buruan, sebelum kehabisan! #Promo #DiskonGede #Sale'},
 {'post_id': 'HT1988',
  'post_time': '16/02/2025, 07:34:18',
  'origin_id': 'admin_3',
  'message_text': 'BIG SALE ALERT! Diskon hingga 50% untuk semua produk kami, hanya sampai hari Minggu ini! Buruan, sebelum kehabisan! #Promo #DiskonGede #Sale'},
 {'post_id': 'ZG9610',
  'post_time': '16/02/2025, 08:07:59',
  'origin_id': 'admin_2',
  'message_text': 'Last Day Promo! Ini hari terakhir buat nikmati potongan harga spesial dari kami! Kapan lagi belanja murah kayak gini? Cek sekarang di #LastChance #PromoAkhir'},
 {'post_id': 'XL4964',
  'post_time': '16/02/2025, 10:18:11',
  'origin_id': 'admin_2',
  'message_text': 'Last Day Promo! Ini hari terakhir buat nikmati potongan harga spesial dari kami! Kapan lagi belanja murah kayak gini? Cek s

In [ ]:
# Step 2 : Transform (Take the first post from each admin then sort by admin name)

## Change the `post_time` column data type to datetime so that it can be sorted
admin_df = pd.DataFrame(results)
admin_df['post_time'] = pd.to_datetime(admin_df['post_time'], format='%d/%m/%Y, %H:%M:%S')

## Get first post per admin based on time
admin_first_posts_df = (
    admin_df.sort_values(by='post_time')
               .groupby('origin_id', as_index=False)
               .first()
               .sort_values(by='origin_id')
               .reset_index(drop=True)
)
admin_first_posts_df

,origin_id,post_id,post_time,message_text
0,admin_1,ER3020,2025-02-16 06:28:12,BIG SALE ALERT! Diskon hingga 50% untuk semua ...
1,admin_2,ZG9610,2025-02-16 08:07:59,Last Day Promo! Ini hari terakhir buat nikmati...
2,admin_3,HT1988,2025-02-16 07:34:18,BIG SALE ALERT! Diskon hingga 50% untuk semua ...


In [ ]:
# Step 3 : Load (Create table `admin_first_message` and insert the data)

## Create table `admin_first_message`
cur.execute("""
    CREATE TABLE IF NOT EXISTS admin_first_message (
        post_id VARCHAR(6),
        post_time TIMESTAMP,
        origin_id VARCHAR(10),
        message_text TEXT
    );
""")

## Insert the data
for index, row in admin_first_posts_df.iterrows():
    print('Insert data - row : ', index)
    cur.execute(
        """
        INSERT INTO admin_first_message (post_id, post_time, origin_id, message_text)
        VALUES (%s, %s, %s, %s);
        """,
        (row['post_id'], row['post_time'], row['origin_id'], row['message_text']))

conn.commit()

Insert data - row :  0
Insert data - row :  1
Insert data - row :  2


### C.4 - Table remaining_token_usage

The `token` field is used for authentication when interacting with the company’s social media accounts. Each `token` can only be used up to 15 times per day. Create a table named `remaining_token_usage` to store the remaining number of times each token can be used per day, based on the dataset. Example table format:

| token | post_date | remaining_quota |
| --- | --- | --- |
| token_1 | 2025-02-16 | 8 |
| token_2 | 2025-02-16 | 1 |
| token_3 | 2025-02-16 | 7 |

In [ ]:
# Step 1 : Extract (Find all posts)

results = coll_raw_data.find(
    {},
    {'_id': 0, 'post_time':1, 'conn_info.token': 1})

results = list(results)
results

[{'post_time': '16/02/2025, 06:28:12', 'conn_info': {'token': '229a6669'}},
 {'post_time': '16/02/2025, 07:34:18', 'conn_info': {'token': '03f2395f'}},
 {'post_time': '16/02/2025, 08:07:59', 'conn_info': {'token': '229a6669'}},
 {'post_time': '16/02/2025, 10:18:11', 'conn_info': {'token': '03f2395f'}},
 {'post_time': '16/02/2025, 12:51:48', 'conn_info': {'token': '229a6669'}},
 {'post_time': '16/02/2025, 13:02:44', 'conn_info': {'token': '03f2395f'}},
 {'post_time': '16/02/2025, 13:03:02', 'conn_info': {'token': '03f2395f'}},
 {'post_time': '16/02/2025, 13:56:50', 'conn_info': {'token': '03f2395f'}},
 {'post_time': '16/02/2025, 14:18:27', 'conn_info': {'token': '21b7c13c'}},
 {'post_time': '16/02/2025, 15:03:35', 'conn_info': {'token': '4eededee'}},
 {'post_time': '16/02/2025, 15:26:16', 'conn_info': {'token': '229a6669'}},
 {'post_time': '16/02/2025, 15:36:16', 'conn_info': {'token': '21b7c13c'}},
 {'post_time': '16/02/2025, 15:48:39', 'conn_info': {'token': '03f2395f'}},
 {'post_time

In [ ]:
# Step 2 : Transform (Get `token` from each document)

## Get the unique date and the token
token_df = pd.DataFrame(results)
token_df['post_time'] = pd.to_datetime(token_df['post_time'], format='%d/%m/%Y, %H:%M:%S')
token_df['post_date'] = token_df['post_time'].dt.date
token_df['token'] = [row['token'] for row in token_df['conn_info']]

## Calculate the usage per day
token_usage_df = (
    token_df.groupby(['token', 'post_date'])
    .size()
    .reset_index(name='used_count')
)

# Maksimum 10 per token per hari
token_usage_df['remaining_quota'] = 15 - token_usage_df['used_count']

# Final DataFrame
remaining_token_usage_df = token_usage_df[['token', 'post_date', 'remaining_quota']]
remaining_token_usage_df

,token,post_date,remaining_quota
0,03f2395f,2025-02-16,4
1,21b7c13c,2025-02-16,12
2,229a6669,2025-02-16,11
3,4eededee,2025-02-16,13


In [ ]:
# Step 3 : Load (Create `remaining_token_usage` and insert the data)

## Create table `remaining_token_usage`
cur.execute("""
    CREATE TABLE IF NOT EXISTS remaining_token_usage (
        post_date TIMESTAMP,
        token VARCHAR(10),
        remaining_quota INTEGER
    );
""")

## Insert the data
for index, row in remaining_token_usage_df.iterrows():
    print('Insert data - row : ', index)
    cur.execute(
        """
        INSERT INTO remaining_token_usage (post_date, token, remaining_quota)
        VALUES (%s, %s, %s)
        """,
        (row['post_date'], row['token'], row['remaining_quota']))

conn.commit()

Insert data - row :  0
Insert data - row :  1
Insert data - row :  2
Insert data - row :  3


### C.5 - Close the connection

In [ ]:
# Close the connection

cur.close()
conn.close()
client.close()